# Inference

## Dependencies and imports

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## Load fintuned model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("VerbACxSS/sempl-it-umt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("VerbACxSS/sempl-it-umt5-small")
model.to('cuda')

UMT5ForConditionalGeneration(
  (shared): Embedding(256384, 512)
  (encoder): UMT5Stack(
    (embed_tokens): Embedding(256384, 512)
    (block): ModuleList(
      (0-7): 8 x UMT5Block(
        (layer): ModuleList(
          (0): UMT5LayerSelfAttention(
            (SelfAttention): UMT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): UMT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): UMT5LayerFF(
            (DenseReluDense): UMT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=Fal

## Load datasets

In [5]:
train_df = pd.read_csv('../dataset/train.tsv', sep='\t', encoding='utf-8')
test_df = pd.read_csv('../dataset/test.tsv', sep='\t', encoding='utf-8')
val_df = pd.read_csv('../dataset/val.tsv', sep='\t', encoding='utf-8')

## Utilities

In [6]:
PREFIX = "semplifica: "
TOKENIZER_MAX_LENGHT = 1024

def predict(samples_array, batch_size=16):
  prompts = [PREFIX + s for s in samples_array]
  batched_prompt = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

  outputs = []
  for batch in tqdm(batched_prompt):
    x = tokenizer(batch, max_length=TOKENIZER_MAX_LENGHT, truncation=True, padding=True, return_tensors='pt').input_ids.to('cuda')
    y =  model.generate(x, max_new_tokens=TOKENIZER_MAX_LENGHT)
    decoded = tokenizer.batch_decode(y, skip_special_tokens=True, clean_up_tokenization_spaces=True, max_length=TOKENIZER_MAX_LENGHT, truncation=True)
    outputs.extend(decoded)
  return outputs

## Random predictions

In [7]:
for s in test_df.sample(10).to_dict(orient='records'):
  output = predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\noracle: ", s['oracle_text'])
  print("\nmodel: ", output)
  print('----------------')

100%|██████████| 1/1 [00:03<00:00,  3.27s/it]



original:  Vi è evidenza del monitoraggio dell'efficacia delle attività di formazione e coinvolgimento del personale per la sicurezza della struttura e dell'ambiente di lavoro, anche attraverso dimostrazioni, simulazioni e altri metodi idonei opportunamente documentate.

oracle:  Si controlla se le attività di formazione e coinvolgimento del personale per la sicurezza sul lavoro funzionano bene. Questo controllo avviene attraverso dimostrazioni, simulazioni e altri metodi che vengono registrati.

model:  Si controlla se le attività di formazione e coinvolgimento del personale sono efficaci, anche attraverso dimostrazioni, simulazioni e altri metodi che sono stati documentati.
----------------


100%|██████████| 1/1 [00:01<00:00,  1.37s/it]



original:  Nel corso del 2021, Gestione Ambientale è altresì diventata anche "società di ingegneria" ai sensi del d.lgs. n. 50/2016 e del Decreto del Ministero delle Infrastrutture 2 dicembre 2016, n. 263, in grado di svolgere servizi di progettazione, consulenza tecnica e gestione cantieri per qualsiasi soggetto pubblico e privato.

oracle:  Nel 2021, la società Gestione Ambientale ha ottenuto la qualifica di "società di ingegneria", che le consente di svolgere servizi di progettazione, consulenza tecnica e gestione cantieri per enti pubblici e privati, in conformità alle normative vigenti.

model:  Nel 2021, Gestione Ambientale è diventata una società di ingegneria, come previsto dal decreto legislativo n. 50/2016 e del Decreto del Ministero delle Infrastrutture n. 263. Questo è stato fatto per progettare, consulenza tecnica e gestire cantieri per qualsiasi soggetto pubblico e privato.
----------------


100%|██████████| 1/1 [00:00<00:00,  2.88it/s]


...


original:  Iscrizione Albo Gestori rifiuti Cat. 1 Classe B per la Gestione dei servizi di raccolta rifiuti e di igiene urbana Comunale con integrazione della Classe B per le attività di spazzamento e gestione di centri di raccolta comunali;

oracle:  Iscrizione all'elenco dei gestori dei rifiuti categoria 1, classe B per gestire la raccolta dei rifiuti e la pulizia delle strade in città, e anche per gestire i centri di raccolta dei rifiuti della città.

model:  Iscrizione del Albo Gestori rifiuti per gestire i servizi di raccolta rifiuti e pulizia della città, con l'integrazione della Classe B per gestire i centri di raccolta comunali.
----------------


100%|██████████| 1/1 [00:00<00:00,  5.01it/s]



original:  Vocabolario comune per gli appalti

oracle:  Parole importanti per fare contratti per lavori o servizi.

model:  Il nome comune per gli appalti
----------------


100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


original:  Avvisi di saldo/conguaglio, che liquidano la tassa dovuta per l'anno d'imposta ed addebitano il netto da versare detraendo gli acconti pagati ed eventuali rimborsi/compensazioni disposte dall'ufficio; sono recapitati all'indirizzo di residenza di ogni singolo contribuente tramite posta ordinaria, mail o PEC;

oracle:  Viene inviato un avviso di saldo o conguaglio per pagare le tasse dovute per l'anno fiscale. Questo avviso indica l'importo che devi ancora pagare, dopo aver sottratto i pagamenti anticipati e eventuali rimborsi. Può essere recapitato a casa tramite posta ordinaria, via email o tramite Posta Elettronica Certificata (PEC).

model:  Avvisi di saldo o conguaglio che liquidano la tassa dovuta per l'anno d'imposta e addebiscono il netto da pagare, come ad esempio i pagamenti pagati e eventuali rimborsi o compensazioni stabiliti dall'ufficio. Questi avvisi devono essere inviati all'indirizzo di residenza di ogni contribuente tramite posta ordinaria, email o PEC.
---

## Run all predictions

In [8]:
train_df['model_text'] = predict(train_df['original_text'].tolist(), batch_size=32)
test_df['model_text'] = predict(test_df['original_text'].tolist(), batch_size=32)
val_df['model_text'] = predict(val_df['original_text'].tolist(), batch_size=32)

100%|██████████| 58/58 [05:26<00:00,  5.63s/it]


## Save simplified datasets

In [9]:
train_df.to_csv('./train_simplified.tsv', index=False, sep='\t')
test_df.to_csv('./test_simplified.tsv', index=False, sep='\t')
val_df.to_csv('./val_simplified.tsv', index=False, sep='\t')